In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import logging
from tqdm import tqdm
import math
import os
import torch_optimizer as optim
from transformers import AdamW
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPTNeoForCausalLM
from transformers.optimization import get_linear_schedule_with_warmup

In [ ]:
#set dataset and tokenizer
datasets = load_dataset('GEM/FairytaleQA')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8548 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1025 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1007 [00:00<?, ? examples/s]

In [ ]:
#create labels (taget text, same as input)
def create_labels(inputs):
    labels=[]
    for ids,attention_mask in zip(inputs['input_ids'],inputs['attention_mask']):
        label=ids.copy()
        real_len=sum(attention_mask)
        padding_len=len(attention_mask)-sum(attention_mask)
        label[:]=label[:real_len]+[-100]*padding_len
        labels.append(label)
    inputs['labels']=labels

In [ ]:
#class dataset
class Dataset_class:
  def __init__(self, inputs):
    self.ids = inputs['input_ids']
    self.att_mask = inputs['attention_mask']
    self.labs=inputs['labels']

  def __len__(self):
    return len(self.ids)

  def __getitem__(self, item):
    return [torch.tensor(self.ids[item], dtype=torch.long),
            torch.tensor(self.att_mask[item], dtype=torch.long),
            torch.tensor(self.labs[item], dtype=torch.long)]

In [ ]:
#preprocess data
def preprocess_data(dataset, tokenizer):
  #set dataset columns
  train_dataset = datasets['train'].remove_columns(['references', 'story_name', 'answer', 'question', 'gem_id', 'target', 'local_or_sum', 'attribute', 'ex_or_im'])
  train_dataset = train_dataset.rename_column('content', 'text')

  valid_dataset = datasets['validation'].remove_columns(['references', 'story_name', 'answer', 'question', 'gem_id', 'target', 'local_or_sum', 'attribute', 'ex_or_im'])
  valid_dataset = valid_dataset.rename_column('content', 'text')

  test_dataset = datasets['test'].remove_columns(['references', 'story_name', 'answer', 'question', 'gem_id', 'target', 'local_or_sum', 'attribute', 'ex_or_im'])
  test_dataset = test_dataset.rename_column('content', 'text')

  #tokenize data
  tokenized_train = tokenizer(train_dataset["text"], padding=True,truncation=True, max_length=512)
  tokenized_valid = tokenizer(valid_dataset["text"], padding=True,truncation=True, max_length=512)
  tokenized_test = tokenizer(test_dataset["text"], padding=True,truncation=True, max_length=512)

  #add labels
  create_labels(tokenized_train)
  create_labels(tokenized_valid)
  create_labels(tokenized_test)

  traindataset = Dataset_class(tokenized_train)
  train_dataloader = torch.utils.data.DataLoader(traindataset, shuffle=True, batch_size=4)

  validdataset = Dataset_class(tokenized_valid)
  valid_dataloader = torch.utils.data.DataLoader(validdataset, shuffle=False, batch_size=4)

  testdataset = Dataset_class(tokenized_test)
  test_dataloader = torch.utils.data.DataLoader(testdataset, shuffle=False, batch_size=4)

  return train_dataloader, valid_dataloader, test_dataloader


In [ ]:
#preprocess dataset
train_dataloader, valid_dataloader, test_dataloader = preprocess_data(datasets, tokenizer)

In [ ]:
#download model
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.to('cuda')
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
#evaluate
def evaluate_model(model, test_set, print_loss=False):
  eval_loss = []
  for inputs in tqdm(test_set, desc="eval test"):
    ids, mask, labs = inputs
    ids = ids.to('cuda')
    mask = mask.to('cuda')
    labs = labs.to('cuda')

    with torch.no_grad():
      output = model(input_ids = ids, attention_mask = mask,labels = labs)
      batch_loss = output[0]
    eval_loss += [batch_loss.cpu().item()]
  eval_loss = np.mean(eval_loss)
  perplexity = math.exp(eval_loss)

  if print_loss:
    print("Average loss on evaluation set = ", eval_loss)

  print('\nPerplexity on evaluation set: ', perplexity)

In [ ]:
#generate story
def generate_story(start, k=0, p=0.9, output_length=300, temperature=1, num_return_sequences=3, repetition_penalty=1.0):
  texts = []

  encoded_start = tokenizer.encode(start, add_special_tokens = False, return_tensors = "pt")
  model.to('cpu')
  model.eval()
  output_sequences = model.generate(input_ids = encoded_start, max_length = output_length, temperature = temperature, top_k = k, top_p = p, repetition_penalty = repetition_penalty, do_sample = True, num_return_sequences = num_return_sequences)
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()
  print("text generated")
  for _ , generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces = True)
    # Remove all text after eos token
    text = text[: text.find(tokenizer.eos_token)]
    print(text, "\n")
    texts.append(text)
  return texts

In [ ]:
#set parameters
class Hparams:
  def __init__(self, train_dataloader, tot_epochs=2, learning_rate=5e-5):
    self.tot_epochs = tot_epochs
    self.training_steps_per_epoch = len(train_dataloader)
    self.total_num_training_steps = int(self.training_steps_per_epoch * tot_epochs)
    self.learning_rate = learning_rate
    self.warmup_steps = int(self.total_num_training_steps * 0.1)


params = Hparams(train_dataloader)
optimizer = torch.optim.AdamW(model.parameters(), lr=params.learning_rate)
#optimizer = AdamW(model.parameters(), lr=params.learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=params.warmup_steps, num_training_steps=params.total_num_training_steps) #to adjust the learning rate during training

In [ ]:
#evaluate pre training
print("Evaluate pre training")
evaluate_model(model, test_dataloader, print_loss=False)

Evaluate pre training


eval test: 100%|██████████| 252/252 [00:49<00:00,  5.08it/s]

Perplexity on evaluation set:  36.34647491872336


In [ ]:
#training
def training(model, train_dataloader, valid_dataloader, params, optimizer, scheduler):
  print("Tot epochs: ", params.tot_epochs)

  for epoch in range(params.tot_epochs):
    print("Start epoch", epoch + 1)

    train_loss=0
    epoch_iterator = tqdm(train_dataloader, desc='Epoch iteration')
    model.train()
    model.zero_grad()
    for _, inputs in enumerate(epoch_iterator):
      ids, mask, labs = inputs
      ids = ids.to('cuda')
      mask = mask.to('cuda')
      labs = labs.to('cuda')
      output = model(input_ids = ids, attention_mask = mask,labels = labs)
      batch_loss = output[0]
      batch_loss.backward()
      optimizer.step()
      scheduler.step()
      model.zero_grad()
      train_loss += batch_loss.item()
      epoch_iterator.set_description('(batch loss=%g)' % batch_loss.item())
    print("Average train loss = ", train_loss/params.training_steps_per_epoch, " [Epoch: ", epoch+1, "]")

    print("Evaluate on Validation set")
    evaluate_model(model, valid_dataloader, print_loss = True)

  return model

In [ ]:
#do training
model = training(model, train_dataloader, valid_dataloader, params, optimizer, scheduler)

Tot epochs:  2
Start epoch 1


(batch loss=3.46063):   4%|▍         | 92/2137 [00:53<20:15,  1.68it/s]

In [ ]:
#save model
path_model = "/content/drive/MyDrive/EAI/Results/story_gpt2_2epochs.pt"
torch.save(model, path_model)

In [ ]:
#evaluate post training
print("Evaluate post training")
evaluate_model(model, test_dataloader, print_loss=False)

In [ ]:
texts = generate_story(prompt,target)